In [7]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0 MB 10 kB/s s eta 0:00:01     |██████████▎                     | 228.6 MB 62.9 MB/s eta 0:00:08 eta 0:00:03
     |████████████████████████████████| 5.9 MB 60.1 MB/s eta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.6.0
    Uninstalling torch-1.6.0:
      Successfully uninstalled torch-1.6.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.7.0
    Uninstalling torchvision-0.7.0:
      Successfully uninstalled torchvision-0.7.0


In [3]:
!pip install mmcv

     |████████████████████████████████| 239 kB 7.0 MB/s eta 0:00:01
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
     |████████████████████████████████| 190 kB 20.1 MB/s eta 0:00:01
  Created wheel for mmcv: filename=mmcv-1.1.4-cp37-cp37m-linux_x86_64.whl size=403074 sha256=decaf2e5ec505606f6048c1fffdf15e69dee17c1a84f688f685c57c2ee129782
  Stored in directory: /home/hongky/.cache/pip/wheels/bc/72/bc/1a7ea2da9f9f1b5a0240332d176f9784c75f2e3cdd13ef0a78
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44619 sha256=9a60c403aa27a2b5513376126bdaf5534cbd69f4d0ccd11e4c2ade1bc0161ecb
  Stored in directory: /home/hongky/.cache/pip/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653
Successfully built mmcv pyyaml


In [3]:
!export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/AvaStore/opt/local/cuda-10.2/lib64
!export PATH=${PATH}:/AvaStore/opt/local/cuda-10.2/bin

In [4]:
!pip install dotmap

  Using cached dotmap-1.3.17-py3-none-any.whl (10 kB)


In [4]:
!python main.py --arch MobileNetV2

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
100%|██████████████████████▉| 170385408/170498071 [00:14<00:00, 14360564.46it/s]Extracting data.cifar10/cifar-10-python.tar.gz to data.cifar10
Done eval model: MobileNetV2(
  (features): Sequential(
    (0): conv_bn_relu(
      (convbn): Sequential(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU6(inplace=True)
      )
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (dw_conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (dw_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dw_relu): ReLU6(inplace=True)
        (project_conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (project_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=


  0%|                                                   | 0/391 [00:00<?, ?it/s]
170500096it [00:29, 14360564.46it/s]                                            
 49%|████████████████████▏                    | 193/391 [13:04<13:03,  3.96s/it]


 99%|████████████████████████████████████████▌| 387/391 [26:15<00:15,  3.98s/it]


100%|█████████████████████████████████████████| 391/391 [26:30<00:00,  4.07s/it]

  0%|                                                    | 0/40 [00:00<?, ?it/s]/home/hongky/.conda/envs/tf_tutorial/lib/python3.7/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))

100%|███████████████████████████████████████████| 40/40 [01:43<00:00,  2.59s/it]

Epoch: 0 Test set: Average loss: 1.8129, Accuracy: 3230/10000 (32.3%)

/home/hongky/.conda/envs/tf_tutorial/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use

In [1]:
from __future__ import print_function
import argparse
import numpy as np
import os
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from models import *
from tqdm import tqdm
from models.slimmableops import bn_calibration_init
import USconfig as FLAGS
import random

In [2]:
torch.cuda.is_available()

True

In [7]:
from dotmap import DotMap

args = DotMap()

args.dataset = 'cifar10'
args.batch_size=128
args.test_batch_size=256
args.epochs = 160
args.start_epoch=0
args.lr = 0.2
args.momentum=0.9
args.weight_decay=1e-4
args.resume=''
args.no_cuda=False
args.seed=1
args.save='checkpoints'
args.arch='MobileNetV2'
args.sr=True
args.s=0.0001
args.test=True


args.cuda = not args.no_cuda and torch.cuda.is_available()

In [8]:

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)


savepath = os.path.join(args.save, args.arch, 'sr' if args.sr else 'nosr')
if not os.path.exists(savepath):
    os.makedirs(savepath)
kwargs = {'num_workers': 4, 'pin_memory': True} if args.cuda else {}
if args.dataset == 'cifar10':
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('data.cifar10', train=True, download=True,
                         transform=transforms.Compose([
                             transforms.RandomCrop(32, padding=4),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(),
                             transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                         ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('data.cifar10', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)
else:
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100('./data.cifar100', train=True, download=True,
                          transform=transforms.Compose([
                              transforms.Pad(4),
                              transforms.RandomCrop(32),
                              transforms.RandomHorizontalFlip(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                          ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100('./data.cifar100', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

Files already downloaded and verified


In [9]:
model = eval(args.arch)(input_size=32)
print('Done eval model:', model)
if args.cuda:
    model.cuda()
best_prec1 = -1
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {}) Prec1: {:f}"
              .format(args.resume, checkpoint['epoch'], best_prec1))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

Done eval model: MobileNetV2(
  (features): Sequential(
    (0): conv_bn_relu(
      (convbn): Sequential(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU6(inplace=True)
      )
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (dw_conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (dw_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dw_relu): ReLU6(inplace=True)
        (project_conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (project_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (expand_conv): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (expand_bn)

In [10]:

def updateBN():
    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.weight.grad.data.add_(args.s * torch.sign(m.weight.data))  # L1


def train():
    model.train()
    avg_loss = 0.
    train_acc = 0.
    for batch_idx, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        if args.sr:
            updateBN()
        optimizer.step()
def trainUS():
    max_width = max(FLAGS.width_mult_list)
    min_width = min(FLAGS.width_mult_list)

    model.train()
    for batch_idx, (data, target) in tqdm(enumerate(train_loader),total=len(train_loader)):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        ###
        widths_train = []
        for _ in range(getattr(FLAGS, 'num_sample_training', 2) - 2):
            widths_train.append(
                random.uniform(min_width, max_width))
        widths_train = [max_width, min_width] + widths_train
        # widths_train = [min_width]
        for width_mult in widths_train:
            # TODO :add inplace distillation
            model.apply(lambda m: setattr(
                m, 'width_mult',
                width_mult))
                # always track largest model and smallest model
            output = model(data)
            loss = F.cross_entropy(output, target)
            loss.backward()
        ###
        optimizer.step()

def test(epoch,test_width=1.0,recal=False):
    model.eval()
    test_loss = 0
    correct = 0
    model.apply(lambda m: setattr(m, 'width_mult',test_width))
    if recal:
        model.apply(bn_calibration_init)
        model.train()
        for idx,(data, target) in enumerate(tqdm(train_loader, total=len(train_loader))):
            if idx==FLAGS.recal_batch:
                break
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
            del output
            
    model.eval()
    for data, target in tqdm(test_loader, total=len(test_loader)):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item()  # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nEpoch: {} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct.item() / float(len(test_loader.dataset))

def export2normal():
    newmodel=MobileNetV2()
    from collections import OrderedDict
    statedic=[]
    for k2,v in model.state_dict().items():
        if 'running' in k2 or 'num_batches_tracked' in k2:
            continue
        statedic.append(v)
    names=[]
    for k1,v1 in newmodel.state_dict().items():
        if 'running' in k1 or 'num_batches_tracked' in k1:
            continue
        names.append(k1)
    newdic=OrderedDict(zip(names,statedic))
    newmodel.load_state_dict(newdic,strict=False)
    torch.save(newmodel.state_dict(),os.path.join(savepath,'trans.pth'))
    print("save transferred ckpt at {}".format(os.path.join(savepath,'trans.pth')))

In [11]:
best_prec1 = 0. if best_prec1 == -1 else best_prec1
scheduler=optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=args.epochs,eta_min=0)

for epoch in range(args.start_epoch, args.epochs):
    if args.arch=='USMobileNetV2':
        trainUS()
        prec1=test(test_width=1.0,recal=False,epoch=epoch)
    else:
        train()
        prec1 = test(epoch=epoch)
    scheduler.step(epoch)
    lr_current = optimizer.param_groups[0]['lr']
    print("currnt lr:{}".format(lr_current))
    is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)
    if is_best:
        ckptfile = os.path.join(savepath, 'model_best.pth.tar')
    else:
        ckptfile = os.path.join(savepath, 'checkpoint.pth.tar')
        
    torch.save({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
        'optimizer': optimizer.state_dict(),
    }, ckptfile)

  0%|          | 0/40 [00:00<?, ?it/s]/home/hongky/.conda/envs/tf_tutorial/lib/python3.7/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
100%|██████████| 40/40 [00:07<00:00,  5.67it/s]
/home/hongky/.conda/envs/tf_tutorial/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



Epoch: 0 Test set: Average loss: 1.7876, Accuracy: 3249/10000 (32.5%)

currnt lr:0.2


100%|██████████| 40/40 [00:06<00:00,  5.73it/s]



Epoch: 1 Test set: Average loss: 1.6081, Accuracy: 4325/10000 (43.2%)

currnt lr:0.1999807240482065


100%|██████████| 40/40 [00:06<00:00,  5.74it/s]


...


Epoch: 157 Test set: Average loss: 0.2787, Accuracy: 9187/10000 (91.9%)

currnt lr:0.00017343898152841764


100%|██████████| 40/40 [00:07<00:00,  5.70it/s]



Epoch: 158 Test set: Average loss: 0.2833, Accuracy: 9185/10000 (91.8%)

currnt lr:7.709637592770991e-05


100%|██████████| 40/40 [00:06<00:00,  5.76it/s]



Epoch: 159 Test set: Average loss: 0.2790, Accuracy: 9188/10000 (91.9%)

currnt lr:1.9275951793518153e-05


In [13]:

if args.arch=='USMobileNetV2':
    export2normal()
    res_acc=[1.0]*len(FLAGS.width_mult_list)
    for idx,width in enumerate(FLAGS.width_mult_list):
        acc=test(width,recal=True)
        res_acc[idx]=acc
        print("Test accuracy for width {} is {}".format(width,acc))
else:
    print("Test accuracy {}".format(test(0)))


100%|██████████| 40/40 [00:06<00:00,  6.00it/s]


Epoch: 0 Test set: Average loss: 0.2790, Accuracy: 9188/10000 (91.9%)

Test accuracy 0.9188


In [14]:
print(ckptfile)
torch.save({
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'best_prec1': best_prec1,
    'optimizer': optimizer.state_dict(),
}, ckptfile)

checkpoints/MobileNetV2/sr/checkpoint.pth.tar


In [16]:
!ls checkpoints/MobileNetV2/sr

0.2.json  0.4.json  0.6.json  checkpoint.pth.tar  model_best.pth.tar
